# TCN Experiments

Theodor Marcu
tmarcu@

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import numpy as np
import json
import pandas as pd
import numpy as np
import time
import string
from matplotlib import pyplot as plt
import tensorflow as tf
import sys
sys.path.append('../')
from brain2brain import utils
from brain2brain import generators
%matplotlib inline

# TF
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model

# TCN
from brain2brain.tcn import TCN
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [3]:
# Read saved paths for training.
saved_paths_676 = utils.get_file_paths("../brain2brain/train_676_norm_files_projects.txt")

In [4]:
# Split the train files into a training and validation set.
train_676, val_676 = utils.split_file_paths(saved_paths_676, 0.8)

In [5]:
# Print the number of samples.
print(utils.get_total_sample_count(train_676, lookback=512 * 5, length = 1, delay = 512 * 0))

15627


In [6]:
total_electrode_count = 114

In [11]:
# The time we look back.
lookback_window = 512 * 5 # 5 seconds
# Length of sequence predicted.
length_pred = 1 # 1 timestep
# Delay between lookback and length.
delay_pred = 0 # No delay.
# Sampling of electrodes.
samples_per_second = 1 # Samples Per Second
timesteps_per_sample = int(lookback_window // samples_per_second)
# Electrodes
electrode_selection = np.arange(0, 114)
electrode_count = len(electrode_selection)

In [12]:
train_676_generator = generators.FGenerator(file_paths = train_676,
                                            lookback=lookback_window, length = length_pred, delay = delay_pred,
                                            batch_size = 128, sample_period = samples_per_second,
                                            electrodes= electrode_selection, shuffle = True, debug=False)

In [13]:
for i in range(len(train_676_generator)):
    print(f"Checking batch {i}.")
    x, y = train_676_generator[i];
    assert not np.any(np.isnan(x))
    assert not np.any(np.isnan(y))

Checking batch 0.
Checking batch 1.
Checking batch 2.
Checking batch 3.
Checking batch 4.
Checking batch 5.
Checking batch 6.
Checking batch 7.
Checking batch 8.
Checking batch 9.
Checking batch 10.
Checking batch 11.
Checking batch 12.
Checking batch 13.
Checking batch 14.
Checking batch 15.
Checking batch 16.
Checking batch 17.
Checking batch 18.
Checking batch 19.
Checking batch 20.
Checking batch 21.
Checking batch 22.
Checking batch 23.
Checking batch 24.
Checking batch 25.
Checking batch 26.
Checking batch 27.
Checking batch 28.
Checking batch 29.
Checking batch 30.
Checking batch 31.
Checking batch 32.
Checking batch 33.
Checking batch 34.
Checking batch 35.
Checking batch 36.
Checking batch 37.
Checking batch 38.
Checking batch 39.
Checking batch 40.
Checking batch 41.
Checking batch 42.
Checking batch 43.
Checking batch 44.
Checking batch 45.
Checking batch 46.
Checking batch 47.
Checking batch 48.
Checking batch 49.
Checking batch 50.
Checking batch 51.
Checking batch 52.
Che

In [14]:
for i in range(len(val_676_generator)):
    print(f"Checking batch {i}.")
    x, y = val_676_generator[i];
    assert not np.any(np.isnan(x))
    assert not np.any(np.isnan(y))

NameError: name 'val_676_generator' is not defined

In [18]:
len(train_676_generator)

122

In [27]:
timesteps_per_sample

640

In [28]:
x, y = train_676_generator[0]
x.shape

(128, 640, 1)

In [29]:
len(range(0, 2560, 4))

640

In [33]:
# TCN
i = Input(shape=(timesteps_per_sample, electrode_count), batch_size=128)
m = TCN()(i)
m = Dense(1)(m)

model = Model(inputs=[i], outputs=[m])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(128, 640, 1)]           0         
_________________________________________________________________
tcn_3 (TCN)                  (128, 64)                 120000    
_________________________________________________________________
dense_2 (Dense)              (128, 1)                  65        
Total params: 120,065
Trainable params: 120,065
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile('adam', 'mae')

In [35]:

# config = model.get_config()
# with open('test.json', 'w') as outfile:
#     json.dump(config, outfile)

In [36]:
len(train_676_generator)

24

In [16]:
# Validation Generator
val_676_generator = generators.FGenerator(file_paths = val_676,
                                          lookback=lookback_window, length = length_pred, delay = delay_pred,
                                          batch_size = 128, sample_period = samples_per_second,
                                          electrodes= electrode_selection, shuffle = False, debug=False)

In [17]:
len(val_676_generator)

41

In [ ]:
targets = []
for i in range(len(val_676_generator)):
    x, y = val_676_generator[i]
    print(y)


[[[ 6.80850435e-001]]

 [[-8.00388833e-001]]

 [[ 3.42477306e-001]]

 [[-7.25584869e-001]]

 [[-8.34340658e-001]]

 [[-3.54622364e-001]]

 [[-3.17130901e-001]]

 [[-1.19223750e+000]]

 [[ 3.61457106e-001]]

 [[-8.24363914e-001]]

 [[ 2.24067667e-001]]

 [[ 1.28550899e+000]]

 [[ 2.01388076e-001]]

 [[ 6.23062734e-001]]

 [[-1.00807341e+000]]

 [[ 2.62762316e-001]]

 [[ 1.02827618e+000]]

 [[-5.76342583e-001]]

 [[-7.12566356e-001]]

 [[-1.54036023e+000]]

 [[ 2.85691138e-001]]

 [[-3.04033279e-001]]

 [[-1.21703555e+000]]

 [[-3.58172945e-003]]

 [[-4.25834523e-001]]

 [[ 2.56174782e-001]]

 [[-1.09176276e-001]]

 [[-4.72789759e-001]]

 [[-4.01034154e-001]]

 [[-1.36943085e+000]]

 [[-3.03808431e-001]]

 [[-7.39045719e-001]]

 [[ 6.23098760e-001]]

 [[ 1.35405159e+000]]

 [[ 1.00826974e+000]]

 [[-4.52888007e-002]]

 [[ 2.84838787e-001]]

 [[ 6.36673544e-001]]

 [[-8.87140739e-001]]

 [[ 3.90701332e-001]]

 [[-5.53074251e-001]]

 [[-7.97855186e-001]]

 [[ 3.29565355e-001]]

 [[ 1.13215

In [54]:
import brain2brain.experiments as experiments
experiments.tcn_experiment2()

Running ExperimentExperiment Description: GRU on 1 electrode for patient 676. No activation.saved_paths: ../brain2brain/train_676_norm_files_projects.txtbatch_size: 128lookback_window: 5slength_prediction: 1 timesteps; 0sdelay_pred: 0 timesteps; 0ssamples_per_second: 512timesteps_per_sample: 2560electrode_count: 1
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(128, 2560, 1)]          0         
_________________________________________________________________
tcn_6 (TCN)                  (128, 64)                 120000    
_________________________________________________________________
dense_5 (Dense)              (128, 1)                  65        
Total params: 120,065
Trainable params: 120,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
24/24 [==============================] - 102s 4s/step - 

KeyboardInterrupt: 

In [49]:
%pwd

'/home/tmarcu/brain2brain/experiments'

In [130]:
# How many samples to draw from val_676_generator 
# in order to see the entire validation set.
val_steps = len(val_676_generator)

In [ ]:
# model.fit_generator(generator=train_676_generator,
#                     steps_per_epoch=len(train_676_generator),
#                     epochs=20,
#                     validation_data=val_676_generator,
#                     validation_steps=val_steps)

In [16]:
model = load_model("/home/tmarcu/brain2brain/experiment_test_676_bs128_tcn_2_model.h5",
                   custom_objects={"TCN": TCN})

In [22]:
train_steps = len(train_676_generator)
val_steps = len(val_676_generator)
p = model.predict_generator(val_676_generator, steps=val_steps,
                            callbacks=None, max_queue_size=100, workers=1,
                            use_multiprocessing=True, verbose=1)

41/41 [==============================] - 63s 2s/step


In [24]:
p[0]

array([nan], dtype=float32)

In [62]:
plt.plot(p)
plt.plot(targets)
plt.title('TCN pred on 1 electrode for patient 676.')
plt.legend(['predicted', 'actual'])

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

In [18]:
targets

NameError: name 'targets' is not defined